## Model 1.0
Powered by [Pietro Terna](https://terna.to.it/)  and [Eleonora Priori](https://www.est-en.unito.it/do/docenti.pl/Alias?eleonora.priori#tab-profilo)


====================================================================================================

## 1

import libs \
MPI init \
context and runner definition \
t(), T(), Tc() function definitions \
random number generator rng creation \
initialization of the parameters from yaml file\
memory allocations to manage ghosts


====================================================================================================

In [1]:
import time
from mpi4py import MPI
from repast4py import context as ctx
import repast4py 
from repast4py import parameters
from repast4py import schedule
from repast4py import core
from typing import Tuple, List, Dict
import numpy as np
import csv


comm = MPI.COMM_WORLD
rank    = comm.Get_rank()
rankNum = comm.Get_size() 

# create the context to hold the agents and manage cross process
# synchronization
context = ctx.SharedContext(comm)

# Initialize the default schedule runner, HERE to create the t() function,
# returning the tick value
runner = schedule.init_schedule_runner(comm)

# tick number
def t():
    return runner.schedule.tick

#Initializes the repast4py.parameters.params dictionary with the model input parameters.
params = parameters.init_params("model1.yaml", "")


#generate random seed
repast4py.random.init(rng_seed=params['myRandom.seed'][rank]) #each rank has a seed
rng = repast4py.random.default_rng 


#timer T()
startTime=-1
def T():
    global startTime
    if startTime < 0:
        startTime=time.time()
    return time.time() - startTime
T()

#cpuTimer Tc()
startCpuTime=-1
def Tc():
    global startCpuTime
    if startCpuTime < 0:
        startCpuTime=time.process_time()
    return time.process_time() - startCpuTime
Tc()

agent_cache={} # dict with uid as keys and agents' tuples as values

===================================================================================================

## 2

create agents' classes and restore_agent function 



===================================================================================================

In [2]:
class Firm(core.Agent):

    TYPE = 0
    
    def __init__(self, local_id: int, rank: int, capital:float, labor:float, duration:int, recipe: float):
        super().__init__(id=local_id, type=Firm.TYPE, rank=rank) #uid
        
        self.capital=capital
        self.labor=labor
        self.duration=duration
        self.recipe = recipe
        self.output=0
        #print("AA", self.uid, self.capital,self.labor,self.duration, self.recipe)
     
    def produce(self, production:float):
        unusedLabor=self.labor
        unusedCapital=self.capital
        
        self.targetProduction=production
        #labor productivity per time unit =0.5
        requiredLabor=self.targetProduction/0.5
        requiredCapital=requiredLabor*(1/self.recipe)
        if requiredLabor > self.labor or requiredCapital > self.capital: 
            self.output=0
        else: 
            self.output=self.targetProduction
            unusedLabor=self.labor-requiredLabor
            unusedCapital=self.capital-requiredCapital
            
        print("firm production", self.uid, self.output, self.labor, self.capital, \
              unusedLabor, unusedCapital, self.recipe, flush=True)
            
        
        
        
        
    

===================================================================================================

## 3

the model

===================================================================================================

In [3]:
class Model:
    
    global params
    PARAMS = params
    
    def __init__(self, params: Dict):
        # the context and the runner are created in step 1        
      
        runner.schedule_event(          0.0,     self.initGhosts) 
        runner.schedule_repeating_event(0.0,  1, self.counter)
        runner.schedule_repeating_event(0.1,  1, self.firmsProducing)
        
        runner.schedule_stop(params['howManyCycles'])
        runner.schedule_end_event(self.finish)
        

        
        # create firm agents  
        for i in range(params['Firm.count'] // rankNum):                                     
            capital= 10 * rng.random()
            labor= 10 * rng.random() 
            duration= round(10 * rng.random())
            recipe= 0.5+rng.random() #L/K [0.5,1.5]
            aFirm =Firm(i, rank, capital, labor, duration, recipe)
            context.add(aFirm)

        
    #initialize ghosts by sending them in the ranks before starting the simulation
    def initGhosts(self):
        pass
    
    #count the cycles number
    def counter(self):
        if int(t()) % params["tickNumber.betweenChecks"] == 0: 
            print("rank", rank, "tick", t(), flush=True)
            
    def firmsProducing(self):
        for aFirm in context.agents(agent_type=0):
            aFirm.produce(1)
            
    #finish
    def finish(self):
        print("THE END!", flush=True)
    
    
    def start(self):
        runner.execute()

=========================================================================================================

## 4

run the model

==========================================================================================================

In [4]:
def run(params: Dict):
    
    model = Model(params) 
    model.start()
    
run(params)

rank 0 tick 0.0
firm production (0, 0, 0) 0 8.084192924390692 0.03068312066030643 8.084192924390692 0.03068312066030643 0.6233048514102364
firm production (1, 0, 0) 0 8.744466237350508 0.9193196936926362 8.744466237350508 0.9193196936926362 1.3759184663677573
firm production (2, 0, 0) 0 2.625694324845931 5.054562548665604 2.625694324845931 5.054562548665604 0.913446560342383
firm production (3, 0, 0) 0 2.3511517604243592 7.891618397028179 2.3511517604243592 7.891618397028179 0.6650467575660453
firm production (4, 0, 0) 0 8.327743626781889 5.848904616153307 8.327743626781889 5.848904616153307 0.7477467693211001
firm production (5, 0, 0) 0 0.46230934245688693 4.007247565838035 0.46230934245688693 4.007247565838035 1.4621737000643793
firm production (6, 0, 0) 0 7.408425685709035 3.951413344293261 7.408425685709035 3.951413344293261 0.7224395100800267
firm production (7, 0, 0) 0 4.886202360996626 8.672624875682235 4.886202360996626 8.672624875682235 0.7865820870730837
firm production (8, 0

firm production (8, 0, 0) 0 3.317397814315912 7.067154245228476 3.317397814315912 7.067154245228476 0.6367563588898187
firm production (9, 0, 0) 0 2.838437062766718 3.9905006250002097 2.838437062766718 3.9905006250002097 0.8036437090601922
firm production (0, 0, 0) 0 8.084192924390692 0.03068312066030643 8.084192924390692 0.03068312066030643 0.6233048514102364
firm production (1, 0, 0) 0 8.744466237350508 0.9193196936926362 8.744466237350508 0.9193196936926362 1.3759184663677573
firm production (2, 0, 0) 0 2.625694324845931 5.054562548665604 2.625694324845931 5.054562548665604 0.913446560342383
firm production (3, 0, 0) 0 2.3511517604243592 7.891618397028179 2.3511517604243592 7.891618397028179 0.6650467575660453
firm production (4, 0, 0) 0 8.327743626781889 5.848904616153307 8.327743626781889 5.848904616153307 0.7477467693211001
firm production (5, 0, 0) 0 0.46230934245688693 4.007247565838035 0.46230934245688693 4.007247565838035 1.4621737000643793
firm production (6, 0, 0) 1 7.40842